In [6]:
!pip install transformers
!pip install datasets
!pip install wandb
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [7]:
import transformers




In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [9]:
df=pd.read_csv('social_media_sentiment_dataset.csv')
df.head()

,ID,Platform,Comment,Sentiment
0,1,Facebook,I love this product! Best purchase ever!,Positive
1,2,YouTube,I hate this group! They should be banned!,Hate Speech
2,3,Facebook,This update is fantastic! Well done.,Positive
3,4,Facebook,This person is a complete joke.,Offensive Language
4,5,Reddit,"You're a waste of space, get lost!",Offensive Language


In [10]:
df_new=df.drop('ID',axis=1)
df_new

,Platform,Comment,Sentiment
0,Facebook,I love this product! Best purchase ever!,Positive
1,YouTube,I hate this group! They should be banned!,Hate Speech
2,Facebook,This update is fantastic! Well done.,Positive
3,Facebook,This person is a complete joke.,Offensive Language
4,Reddit,"You're a waste of space, get lost!",Offensive Language
...,...,...,...
9995,Twitter,"What an idiot, I can't believe they think that.",Offensive Language
9996,Twitter,I have no strong opinions about this.,Neutral
9997,Twitter,"This app keeps crashing, so frustrating.",Negative
9998,Facebook,Just another ordinary day.,Neutral


In [11]:
df_new.isnull().sum()

,0
Platform,0
Comment,0
Sentiment,0


In [12]:
df_new["Sentiemt"]=df_new.rename(columns={"Sentiment":"labels"},inplace=True)

In [13]:
df_new.head()

,Platform,Comment,labels,Sentiemt
0,Facebook,I love this product! Best purchase ever!,Positive,None
1,YouTube,I hate this group! They should be banned!,Hate Speech,None
2,Facebook,This update is fantastic! Well done.,Positive,None
3,Facebook,This person is a complete joke.,Offensive Language,None
4,Reddit,"You're a waste of space, get lost!",Offensive Language,None


In [14]:
df_new.drop('Sentiemt',axis=1,inplace=True)

In [15]:
df_new.head(1)

,Platform,Comment,labels
0,Facebook,I love this product! Best purchase ever!,Positive


In [16]:
label_encoder=LabelEncoder()
df_new['labels']=label_encoder.fit_transform(df_new['labels'])
df_new.head(4)

,Platform,Comment,labels
0,Facebook,I love this product! Best purchase ever!,4
1,YouTube,I hate this group! They should be banned!,0
2,Facebook,This update is fantastic! Well done.,4
3,Facebook,This person is a complete joke.,3


In [17]:
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

{'Hate Speech': np.int64(0), 'Negative': np.int64(1), 'Neutral': np.int64(2), 'Offensive Language': np.int64(3), 'Positive': np.int64(4)}


In [18]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [19]:
# Function to tokenize each comment
def tokenize(example):
    return tokenizer(example["Comment"], truncation=True, padding="max_length", max_length=128)


In [20]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_new[["Comment", "labels"]])
tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_dataset, test_dataset = tokenized_dataset.train_test_split(test_size=0.2).values()

In [23]:
print(f"Training data size: {len(train_dataset)}")
print(f"Test data size: {len(test_dataset)}")

Training data size: 8000
Test data size: 2000


In [24]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


In [25]:
training_args = TrainingArguments(
    output_dir="./results",
  #  evaluation_strategy="epoch",             # must evaluate per epoch for early stopping
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",                   # must save to compare best mode            # important!
    metric_for_best_model="eval_loss",       # or use "accuracy" if you're tracking that
    greater_is_better=False,                 # since lower loss is better
)


In [26]:
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=len(df_new["labels"].unique()))

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
trainer = Trainer(
    model=model,                      # The model to train
    args=training_args,               # The training arguments
    train_dataset=train_dataset,      # The training dataset
    eval_dataset=test_dataset,        # The evaluation dataset
    tokenizer=tokenizer,               # The tokenizer for preprocessing the data

)


<ipython-input-27-5f0d897a8620>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
import wandb
wandb.init(project="bart-sentiment-classification")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tusharguptaindia614 (tusharguptaindia614-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [29]:
# Training the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,0.083800
1000,0.000300
1500,0.000100
2000,0.000100


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning 

TrainOutput(global_step=2000, training_loss=0.02109791376441717, metrics={'train_runtime': 566.6228, 'train_samples_per_second': 28.237, 'train_steps_per_second': 3.53, 'total_flos': 1226777358336000.0, 'train_loss': 0.02109791376441717, 'epoch': 2.0})

In [62]:
import numpy as np

def predic(new_data):
  new_dataset = Dataset.from_dict({"Comment": new_data})
  tokenized_new = new_dataset.map(tokenize)
  predictions = trainer.predict(tokenized_new)
  pred_logits = predictions.predictions[0]
  predicted_labels = np.argmax(pred_logits, axis=1)
  return predicted_labels



In [67]:
print("Enter the Sentence")
var=input();

decoded_predictions = label_encoder.inverse_transform(predic([var]))
print(decoded_predictions)


Enter the Sentence
woww


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

['Positive']


In [32]:
!git clone https://github.com/Tushargupta0408/Sentiment-Analysis-BART-.git

Cloning into 'Sentiment-Analysis-BART-'...
